In [ ]:
import json
import pandas as pd
from pathlib import Path

In [ ]:
df = pd.read_parquet('iNat data 14-Feb-24.parquet')
len(df)

In [ ]:
species = pd.read_csv("iNat data codes - 'Other' species.csv")
species['Species'] = species['Species'].str.split(',')

In [ ]:
species_names = []
for x in species['Genus']:
    for y in species[species['Genus'] == x]['Species']:
        for z in y:
            species_names.append(f'{x} {z}')
species_names

In [ ]:
df = df[(df['scientific_name'].isin(species_names)) & (~df['tracks'].isna())]
len(df)

In [ ]:
with open('project-5-at-2024-05-03-19-50-7bc08b6a.json') as j:
    data = json.load(j)

exclude = [Path(x['data']['image']).parent.name for x in data]
len(exclude)

In [ ]:
# df['image_url'] = df['image_url'].apply(lambda x: Path(x).parent.stem if x else None)
# len(df)

In [ ]:
import_images = df[~df['image_url'].isin(exclude)]
import_images

In [ ]:
import_images.rename(columns={'image_url': 'image', 'url': 'observation_url'}, inplace=True)

In [ ]:
import_images = import_images[['image', 'id', 'observation_url', 'scientific_name']]

In [ ]:
import_images['image'] = import_images['image'].str.replace('https://static.inaturalist.org', 'https://inaturalist-open-data.s3.amazonaws.com')

In [ ]:
import_images['image'] = import_images['image'].str.replace('medium', 'original')

In [ ]:
import requests
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

def is_valid(url):
    try:
        response = requests.head(url)
        return response.status_code == 200
    except Exception as e:
        print(f"Error checking URL {url}: {e}")
        return False

def check_url_parallel(url):
    return is_valid(url)

with Pool(cpu_count()) as p:
    valid_mask = list(tqdm(p.imap(check_url_parallel, import_images['image']), total=len(import_images)))

import_images = import_images[valid_mask]

import_images.reset_index(drop=True, inplace=True)

In [ ]:
import_images.to_csv('import_images_filtered_to_other_signs.csv', index=False)